Nog een keer hetzelfde programma alleen deze keer maak ik het met ctk. Dat is een Python UI library based on Tkinter die een nieuw, modernere widgets heeft.

In [106]:
# import sys
# !{sys.executable} -m pip install ctk

In [107]:
import tkinter as tk
import customtkinter as ctk
import numpy as num
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

In [108]:
class App:
    def __init__(self):
        ctk.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
        # ctk.set_default_color_theme("pink")  # Themes: "blue" (standard), "green", "dark-blue"

        self.app = ctk.CTk()
        self.app.geometry("400x400")
        self.app.title("DataVisualisation App voor Biker")
        
        self.loginAttempts = 0
        self.createLogin()
        
        # wanneer de windows word gesloten sluit hij ook het programma.
        self.app.protocol("WM_DELETE_WINDOW", self.closeApp)
        

    def loginCheck(self):
        # gelezen logingegevens... Super Secure!!!
        creds = pd.read_csv('logingegevens.csv')

        # omzetten naar 2D array. [[username, password], [username, password], enz..]
        credsArr = creds.to_numpy()

        # check login input
        self.username = self.usernameEntry.get()
        self.password = self.passwordEntry.get()

        # loop series van namen en ww
        for arr in credsArr:
            # subset van naam == [0] en ww == [1]
            if arr[0] == self.username:
                if arr[1] == self.password:
                    # username en password zijn correct
                    self.loginOut(True)
                    return
        
        self.loginOut(False)
                
    def loginOut(self, out):
        if out == True:
            self.closeApp()
            m = Main(self.username)
            m.main.mainloop()

        # login pogingen staat op 2. de user heeft namelijk maar 3 pogingen.
        if self.loginAttempts >= 2 and out == False:
            # de reden voor and out == False is omdat als de user 2 keer zijn wachtwoord fout invult en daarna correct triggert hij ook als de 2e conditie er niet staat
            self.closeApp()
            e = ErrorWindow(reden="WW")
            e.window.mainloop()
            
        elif out == False:
            self.loginAttempts = self.loginAttempts + 1
            label = ctk.CTkLabel(master=self.loginWindow, text=f'Logingegevens zijn incorrect.')
            label.pack()

            labelMessage = ctk.CTkLabel(master=self.loginWindow, text=f'Dit is poging {self.loginAttempts}. u heeft nog {3- self.loginAttempts} pogingen')
            labelMessage.pack()
    
    def closeApp(self):
        self.app.quit()
        self.app.destroy()

    def startApp(self):
        self.app.mainloop()

    # maakt het loginscherm
    def createLogin(self):
        self.loginWindow = ctk.CTkFrame(master=self.app)
        self.loginWindow.pack(pady=20, padx=60, fill="both", expand=True)

        self.usernameEntry = ctk.CTkEntry(master=self.loginWindow, placeholder_text="Username:")
        self.usernameEntry.pack(pady=10, padx=10)

        self.passwordEntry = ctk.CTkEntry(master=self.loginWindow, placeholder_text="Password:", show="*")
        self.passwordEntry.pack()

        self.logInButton = ctk.CTkButton(master=self.loginWindow, text="Login!", command=self.loginCheck)
        self.logInButton.pack(pady=10, padx=10)
        
        self.close = ctk.CTkButton(master=self.loginWindow, text="Afsluiten", command=self.closeApp)
        self.close.pack()



In [109]:
class Main:
    def __init__(self, username):
        self.main = ctk.CTk()
        self.main.geometry("600X800")
        self.main.title("DataVisualisation App voor Biker")
        self.createFrames()

        self.selectedDiagram = "none"
        
        self.username = username
        self.getThemeSetting(self.username)

        # voor themes

        # wanneer de windows word gesloten sluit hij ook het programma.
        self.main.protocol("WM_DELETE_WINDOW", self.closeApp)

    def createFrames(self):
        self.frameSide = ctk.CTkFrame(master=self.main, width=400, height=650)
        self.frameSide.pack(expand=False, side="left", fill="y")
        self.frameSide.pack_propagate(False)
        self.fillFrameSide()

        self.frameMain = ctk.CTkFrame(master=self.main, width=850, height=650)
        self.frameMain.pack(expand=False, side="right", fill="y")
        self.frameMain.pack_propagate(False)
        self.fillFrameMain()

    def fillFrameSide(self):
        self.options = ctk.CTkFrame(master=self.frameSide, width=400, height=700)
        # .pack(fill="none", expand=True) => centers widget! options aangemaakt voor style om de options te centeren in de sideFrame
        self.options.pack(fill="none", expand=True, ipady=10)
        self.label = ctk.CTkLabel(master=self.options, text="Kies hieronder data dat je wilt zien")
        # .pack(fill="none", expand=True) => centers widget!
        self.label.pack()

        self.artikelTaartButton = ctk.CTkButton(master=self.options, text="Verdeling van artikelen in een taart diagram", command=self.onArtikelTaartButton)
        self.artikelTaartButton.pack(pady=10, padx=20)

        self.artikelBarButton = ctk.CTkButton(master=self.options, text="Verdeling van artikelen in een bar diagram", command=self.onArtikelBarButton)
        self.artikelBarButton.pack(pady=10, padx=20)

        self.verdelingTaartButton = ctk.CTkButton(master=self.options, text="Verdeling van console uitgaven per jaar in taart diagram", command=self.onReleaseTaartButton)
        self.verdelingTaartButton.pack(pady=10, padx=20)        

        self.verdelingBarButton = ctk.CTkButton(master=self.options, text="Verdeling van console uitgaven per jaar in bar diagram", command=self.onReleaseBarButton)
        self.verdelingBarButton.pack(pady=10, padx=20) 

        self.themeButton = ctk.CTkButton(master=self.options, text="Verander Thema", command=self.changeTheme)
        self.themeButton.pack(pady=10)

        self.close = ctk.CTkButton(master=self.options, text="Afsluiten", command=self.closeApp)
        self.close.pack(pady=10)

    # mainFrame word gevult opn basis van de keuzes links!
    def fillFrameMain(self):
        self.components = ctk.CTkFrame(master=self.frameMain, width=800, height=600)
        self.components.pack(fill="none", expand=True)
        self.components.pack_propagate(False)

        self.label = ctk.CTkLabel(master=self.components, text="Maak een keuze in het menu links!")
        self.label.pack(fill="none", expand=True)

    # button functions
    def onArtikelTaartButton(self):
        self.killOldDiagram(self.selectedDiagram)
        self.selectedDiagram = "taartA"
        self.diagram = Diagram("taartA", "artikel", self.theme)
        self.createWindow(self.diagram.fig)
        self.createInfo("ArtikelTaart")
    
    def onArtikelBarButton(self):
        self.killOldDiagram(self.selectedDiagram)
        self.selectedDiagram = "barA"
        self.diagram = Diagram("barA", "artikel", self.theme)
        self.createWindow(self.diagram.fig)
        self.createInfo("ArtikelBar")

    def onReleaseTaartButton(self):
        self.killOldDiagram(self.selectedDiagram)
        self.selectedDiagram = "taartR"
        self.diagram = Diagram("taartR", "release", self.theme)
        self.createWindow(self.diagram.fig)
        self.createInfo("ReleaseTaart")

    def onReleaseBarButton(self):
        self.killOldDiagram(self.selectedDiagram)
        self.selectedDiagram = "barR"
        self.diagram = Diagram("barR", "release", self.theme)
        self.createWindow(self.diagram.fig)
        self.createInfo("ReleaseBar")
    
    def setTheme(self):
        if self.theme == "dark":
            ctk.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
            ctk.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"
        elif self.theme == "white":
            ctk.set_appearance_mode("light")  # Modes: "System" (standard), "Dark", "Light"
            ctk.set_default_color_theme("dark-blue")  # Themes: "blue" (standard), "green", "dark-blue"    

    def reloadDiagram(self, diaType):
        # first killDiagram voor de oude diagram
        if diaType == "none":
            return
        else:
            self.diagram.killDiagram()
        # losse if statement om de voor het aanmaken van de aangepaste diagram
        if diaType == "taartA":
            self.diagram = Diagram("taartA", "artikel", self.theme)
            self.createWindow(self.diagram.fig)
        elif diaType == "barA":
            self.diagram = Diagram("barA", "artikel", self.theme)
            self.createWindow(self.diagram.fig)
        elif diaType == "taartR":
            self.diagram = Diagram("taartR", "release", self.theme)
            self.createWindow(self.diagram.fig)
        elif diaType == "barR":
            self.diagram = Diagram("barR", "release", self.theme)
            self.createWindow(self.diagram.fig)
        else:
            # voor edgecase
            e = ErrorWindow("Diagram")
            e.window.mainloop()
            self.closeApp
            

    def changeTheme(self):
        # standaard is systeem thema. met deze know wijk je af van de standaard
        if self.theme == "dark":
            ctk.set_appearance_mode("light")  # Modes: "System" (standard), "Dark", "Light"
            ctk.set_default_color_theme("dark-blue")  # Themes: "blue" (standard), "green", "dark-blue"
            self.theme = "white"
            self.reloadDiagram(self.selectedDiagram)
        elif self.theme == "white":
            ctk.set_appearance_mode("dark")  # Modes: "System" (standard), "Dark", "Light"
            ctk.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"
            self.theme = "dark"
            self.reloadDiagram(self.selectedDiagram)

    def createWindow(self, figure):
        self.resetComponents()
        self.canvasFrame = ctk.CTkFrame(master=self.components)
        self.canvasFrame.pack()
        canvas = FigureCanvasTkAgg(figure, self.canvasFrame)
        canvas.draw()

        canvas.get_tk_widget().pack(fill="none", expand=True)
    
    def createInfo(self, info):
        self.textbox = ctk.CTkTextbox(master=self.components, width=500, wrap="word")
        if info == "ArtikelTaart":
            self.textbox.insert(index="0.0", text="Hierboven zie je info over de verdeling tussen het aantal spellen en consoles in een taart diagram")
            self.textbox.pack(pady=10)
        elif info == "ArtikelBar":
            self.textbox.insert(index="0.0", text="Hierboven zie je info over de verdeling tussen het aantal spellen en consoles in een Bar diagram")
            self.textbox.pack(pady=10)
        elif info == "ReleaseTaart":
            self.textbox.insert(index="0.0", text="Hierboven zie je een Taart diagram die laat zie hoeveel consoles er uit kwamen uit welk jaar.")
            self.textbox.pack(pady=10)
        elif info == "ReleaseBar":
            self.textbox.insert(index="0.0", text="Hierboven zie je een Bar diagram die laat zie hoeveel consoles er uit kwamen uit welk jaar.")
            self.textbox.pack(pady=10)

    def getThemeSetting(self, username):
        self.settings = pd.read_csv('UserSettings.csv')
        self.settingsArr = self.settings.to_numpy()
        # nu hebben we een 2d array met alle settings. deze bestaan uit [[username, setting, value], [etc]]
        for set in self.settingsArr:
            if set[0] == self.username:
                self.theme = set[2]
        self.setTheme()
    
    # dient uitgevoerd te worden wanneer er een nieuwe diagram word opgevraagd om de oude handmatig te killen => voorkomt onnodige overbelasting
    def killOldDiagram(self, diagram):
        if diagram != "none":
            self.diagram.killDiagram()

    def resetComponents(self):
        for widget in self.components.winfo_children():
            widget.destroy()


    def closeApp(self):
        self.main.quit()
        self.main.destroy()


Ik heb resetComponents() toegevoegd omdat zodra de user een keuze maakt in het keuzemenu moet main leeg zijn en nieuwe data komen obv de keuze. zie comments over de werking.  <br>
De array winfo_children word gevuld met alle widgets die aanwezig zijn voordat een keuze word gemaakt. de functie destroyStartMain() loopt dan door alle widgets en .destroy() op alle widgets. <br>

```
def resetComponents(self):
    for widget in self.components.winfo_children():
        widget.destroy()
```


In [110]:
class Diagram:
    def __init__(self, diaType, dataType, theme):
        self.diaType = diaType
        self.dataType = dataType
        self.theme = theme

        self.getData(self.dataType)
        self.setValuesDiagram(self.diaType, self.theme)

    def setValuesDiagram(self, diaType, theme):
        # first we check for the theme! if dark then dark colors if white then white colors
        if theme == "dark":
            self.fig.set_facecolor("#3C3C3B")
            self.ax.set_facecolor("#3C3C3B")
            if diaType == "taartR":
                self.ax.pie(self.y, labels = self.xUnique, startangle = 90, autopct='%1.1f%%', textprops={'color':"w"})
                self.ax.set_title("Aantal console releases per jaar", color="white")
            elif diaType == "taartA":
                self.ax.pie(self.val, labels = self.labels, startangle = 90, autopct='%1.1f%%', textprops={'color':"w"})
                self.ax.set_title("Pie chart over de verdeling van alle artikelen!", color="white")
            elif diaType == "barR" or diaType == "barA":
                self.ax.tick_params(axis="both", colors="white")
                if diaType == "barR":
                    self.ax.set_title("Aantal console releases per jaar", color="white")
                    self.ax.bar(self.xUnique, self.y)
                    self.ax.set_ylabel("Aantal", color="white")
                    self.ax.set_xlabel("Jaar", color="white")
                elif diaType == "barA":
                    self.ax.bar(self.labels, self.val)
                    self.ax.set_title("Bar chart over de verdeling tussen consoles en artikelen!", color="white")
        # TODO waneer themes word geimplementeerd
        elif theme == "white":
            if diaType == "taartR":
                self.ax.pie(self.y, labels = self.xUnique, startangle = 90, autopct='%1.1f%%')
                self.ax.set_title("Aantal console releases per jaar")
            elif diaType == "taartA":
                self.ax.pie(self.val, labels = self.labels, startangle = 90, autopct='%1.1f%%')
                self.ax.set_title("Pie chart over de verdeling van alle artikelen!")
            elif diaType == "barR" or diaType == "barA":
                self.ax.tick_params(axis="both")
                if diaType == "barR":
                    self.ax.set_title("Aantal console releases per jaar")
                    self.ax.bar(self.xUnique, self.y)
                    self.ax.set_ylabel("Aantal")
                    self.ax.set_xlabel("Jaar")
                elif diaType == "barA":
                    self.ax.bar(self.labels, self.val)
                    self.ax.set_title("Bar chart over de verdeling tussen consoles en artikelen!")
    
    def getData(self, dataType):
        if dataType == "artikel":
            self.fig, self.ax = plt.subplots()
            self.labels = ["Spellen", "Consoles"]
            colSpel = ["TITEL"]
            colConsole = ["TYPE"]
            dataSpel = pd.read_csv('Artikel.csv', usecols=colSpel, encoding='latin-1')
            dataConsole = pd.read_csv('Artikel.csv', usecols=colConsole, encoding='latin-1')
            dataConsole.dropna(inplace=True)
            dataSpel.dropna(inplace=True)
            self.val = [dataSpel.size, dataConsole.size]
        elif dataType == "release":
            self.fig, self.ax = plt.subplots()
            # self.fig.set_facecolor("#3C3C3B")
            col = ["JAAR_UITGAVE"]
            data = pd.read_csv('console.csv', usecols=col)
            self.xUnique = num.unique(data)
            count = data.value_counts()
            self.y = []
            for i in self.xUnique:
                self.y.append(count.get(i))
    
    def killDiagram(self):
        plt.close(self.fig)


In [111]:
class ErrorWindow:
    def __init__(self, reden):
        self.reden = reden
        self.window = ctk.CTk()
        self.window.title("DataVisualisation App voor Biker")

        if self.reden == "WW":
            self.message = "U heeft 3 keer de verkeerde logingegevens ingevuld. De applicatie sluit nu."
        elif self.reden == "Diagram":
            self.message = "Er is iets mis gegaan bij het laden van de diagram. Maak hier een melding van bi uw leidinggevende!"
        else:
            self.message = "Error is unknown. Maak hier een melding van bij uw leidinggevende"            

        errorMessage = ctk.CTkLabel(master=self.window, text=self.message)
        errorMessage.pack()

        # wanneer de windows word gesloten sluit hij ook het programma.
        self.window.protocol("WM_DELETE_WINDOW", self.closeApp)


    def closeApp(self):
        self.window.quit()
        self.window.destroy()


In [112]:
if __name__ == "__main__":
    x = App()
    x.startApp()

    # x = Main()
    # x.main.mainloop()


Toen ik bezig was met het testen van de applicatie liep ik tegen deze waarschuwing aan.  
  
C:\Users\lucas\AppData\Local\Temp\ipykernel_7504\2656707410.py:62: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`)

Dit is totaal niet efficient... Hier bedacht ik de volgende oplossing op:  
  
```matplotlib.pyplot.close()```
  
nu moet ik deze allen nog implementeren elke keer wanneer ik van diagram wissel of wanneer ik van thema wissel. 

een andere zwakte als het gaat om efficientie is wanneer ik wissel van thema's dan maak ik een hele nieuwe window ipv aanpassen van gegevens.